In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Data Input

In [ ]:
# data load
train = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train

In [ ]:
# data shape
display(train.shape)

In [ ]:
display(train.label.value_counts())

In [ ]:
# data load
test = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
test

In [ ]:
# data shape
display(test.shape)

## data sample
- 0,2,4 digit data use

In [ ]:
# 0, 2, 4 label value 
train_sample = train.loc[train.label.isin([0,2,4])]

# check
display(train_sample.label.value_counts())

In [ ]:
# target define
y = train_sample.label
X = train_sample.drop('label', axis = 1)

# check
display(X, y)

## Dimension Reduction : PCA

In [ ]:
# preprocessing

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
pre_X = sc.fit_transform(X)

# check
display(pre_X)

In [ ]:
# scikit-learn PCA
from sklearn.decomposition import PCA

pca = PCA(n_components = 5)

# fit
pca.fit(pre_X)
# transform
X_pca = pca.transform(pre_X)

display(X_pca.shape, X_pca)

In [ ]:
display('pca variance : ', pca.explained_variance_ratio_)

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

In [ ]:
trace0 = go.Scatter(
    x = X_pca[:,0],
    y = X_pca[:,1],
    mode = 'markers',
    text = y,
    showlegend = False,
    marker = dict(
        size = 8,
        color = y,
        colorscale ='Jet',
        showscale = False,
        line = dict(
            width = 2,
            color = 'rgb(255, 255, 255)'
        ),
        opacity = 0.8
    )
)
data = [trace0]

layout = go.Layout(
    title= 'Principal Component Analysis (PCA)',
    hovermode= 'closest',
    xaxis= dict(
         title= 'First Principal Component',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Second Principal Component',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend= True
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

## Classify Comparison

### Basic

In [ ]:
# target define
y = train.label
X = train.drop('label', axis = 1)

# check
display(X, test, y)

In [ ]:
# preprocessing

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
pre_X = sc.fit_transform(X)
pre_test = sc.transform(test)

# check
display(pre_X,pre_test)

In [ ]:
# train_test split
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(pre_X, y, stratify=y, random_state=42 )

basic_train_shpae = X_train.shape

In [ ]:
# Training time Check

import time

start_time = time.time()

from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators = 400)
lgbm.fit(X_train, y_train)

basic_time = time.time() - start_time

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

pred_valid = lgbm.predict(X_valid)

basic_confusion_valid=confusion_matrix(y_valid, pred_valid)
basic_accuracy = accuracy_score(y_valid, pred_valid)

display(basic_confusion_valid, basic_accuracy)

### PCA

In [ ]:
# target define
y = train.label
X = train.drop('label', axis = 1)

# check
display(X, test, y)

In [ ]:
# preprocessing

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
pre_X = sc.fit_transform(X)
pre_test = sc.transform(test)

# check
display(pre_X,pre_test)

In [ ]:
# scikit-learn PCA
from sklearn.decomposition import PCA

pca = PCA(n_components = 10)

# fit
pca.fit(pre_X)
# transform
X_pca = pca.transform(pre_X)
test_pca = pca.transform(pre_test)

display(X_pca.shape, X_pca)
display(test_pca.shape,test_pca)

In [ ]:
display('pca variance : ', pca.explained_variance_ratio_)

In [ ]:
# train_test split
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_pca, y, stratify=y, random_state=42 )

pca_train_shpae = X_train.shape

In [ ]:
# Training time Check

import time

start_time = time.time()

from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(n_estimators = 400)
lgbm.fit(X_train, y_train)

pca_time = time.time() - start_time

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

pred_valid = lgbm.predict(X_valid)

pca_confusion_valid = confusion_matrix(y_valid, pred_valid)
pca_accuracy = accuracy_score(y_valid, pred_valid)

display(pca_confusion_valid, pca_accuracy)

## Comparison Models

In [ ]:
print("Basic LGBM Model Train Shape : ", basic_train_shpae)
print("Basic LGBM Model Training Time: ",basic_time)
print("Basic LGBM Model Evaluation : ", basic_accuracy)
display(basic_confusion_valid)

print()
print("==========================================================================================================")
print()

print("PCA LGBM Model Train Shape : ", pca_train_shpae)
print("PCA LGBM Model Training Time: ",pca_time)
print("PCA LGBM Model Evaluation : ", pca_accuracy)
display(pca_confusion_valid)


## Dimension Reduction : LDA

In [ ]:
# 6,8,5 label value 
train_sample = train.loc[train.label.isin([6,8,5])]

# check
display(train_sample.label.value_counts())

In [ ]:
# target define
y = train_sample.label
X = train_sample.drop('label', axis = 1)

# check
display(X, y)

In [ ]:
# preprocessing

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
pre_X = sc.fit_transform(X)

# check
display(pre_X)

In [ ]:
# scikit-learn LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components = 2)

# fit
lda.fit(pre_X,y)
# transform
X_lda = lda.transform(pre_X)

display(X_lda.shape, X_lda)

In [ ]:
display('lda variance : ', lda.explained_variance_ratio_)

In [ ]:
trace0 = go.Scatter(
    x = X_lda[:,0],
    y = X_lda[:,1],
    mode = 'markers',
    text = y,
    showlegend = False,
    marker = dict(
        size = 8,
        color = y,
        colorscale ='Jet',
        showscale = False,
        line = dict(
            width = 2,
            color = 'rgb(255, 255, 255)'
        ),
        opacity = 0.8
    )
)
data = [trace0]

layout = go.Layout(
    title= 'Linear Discriminant Analysis (LDA)',
    hovermode= 'closest',
    xaxis= dict(
         title= 'First Linear Discriminant',
        ticklen= 5,
        zeroline= False,
        gridwidth= 2,
    ),
    yaxis=dict(
        title= 'Second Linear Discriminant',
        ticklen= 5,
        gridwidth= 2,
    ),
    showlegend= True
)


fig = dict(data=data, layout=layout)
py.iplot(fig, filename='styled-scatter')

## Classify Comparison

### Basic

In [ ]:
# target define
y = train.label
X = train.drop('label', axis = 1)

# check
display(X, test, y)

In [ ]:
# preprocessing

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
pre_X = sc.fit_transform(X)
pre_test = sc.transform(test)

# check
display(pre_X,pre_test)

In [ ]:
# train_test split
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(pre_X, y, stratify=y, random_state=42 )

basic_train_shpae = X_train.shape

In [ ]:
# Training time Check

import time
from sklearn.neighbors import KNeighborsClassifier

start_time = time.time()

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

basic_time = time.time() - start_time

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

pred_valid = knn.predict(X_valid)

basic_confusion_valid=confusion_matrix(y_valid, pred_valid)
basic_accuracy = accuracy_score(y_valid, pred_valid)

display(basic_confusion_valid, basic_accuracy)

### LDA

In [ ]:
# target define
y = train.label
X = train.drop('label', axis = 1)

# check
display(X, test, y)

In [ ]:
# preprocessing

from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
pre_X = sc.fit_transform(X)
pre_test = sc.transform(test)

# check
display(pre_X,pre_test)

In [ ]:
# scikit-learn LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

lda = LDA(n_components = 5)

# fit
lda.fit(pre_X, y)
# transform
X_lda = lda.transform(pre_X)

display(X_lda.shape, X_lda)

In [ ]:
display('lda variance : ', lda.explained_variance_ratio_)

In [ ]:
# train_test split
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_lda, y, stratify=y, random_state=42 )

lda_train_shpae = X_train.shape

In [ ]:
# Training time Check

import time
from sklearn.neighbors import KNeighborsClassifier

start_time = time.time()

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)

lda_time = time.time() - start_time

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score

pred_valid = knn.predict(X_valid)

lda_confusion_valid = confusion_matrix(y_valid, pred_valid)
lda_accuracy = accuracy_score(y_valid, pred_valid)

display(lda_confusion_valid, lda_accuracy)

## Comparison Models

In [ ]:
print("Basic KNN Model Train Shape : ", basic_train_shpae)
print("Basic KNN Model Training Time: ",basic_time)
print("Basic KNN Model Evaluation : ", basic_accuracy)
display(basic_confusion_valid)

print()
print("==========================================================================================================")
print()

print("LDA KNN Model Train Shape : ", lda_train_shpae)
print("LDA KNN Model Training Time: ",lda_time)
print("LDA KNN Model Evaluation : ", lda_accuracy)
display(lda_confusion_valid)
